# scgpt + molgpt pipeline....

## installs

In [1]:
!pip install anndata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install scanpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 72.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [3]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 53.3 MB/s eta 0:00:00


In [4]:
!pip install deeplife-mlinfra


ERROR: Could not find a version that satisfies the requirement deeplife-mlinfra (from versions: none)
ERROR: No matching distribution found for deeplife-mlinfra


## scripts

In [17]:
%%writefile scgpt_smiles_model.py

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import anndata
import scanpy as sc
import os
import numpy as np
from typing import Dict, List, Optional, Tuple, Union
import math
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

from molgpt_loader import load_mol_gpt_model


# Model Components
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0)]

class DoublePositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        # Use the full embedding dimension divided into two halves
        self.d_model = d_model
        half_dim = d_model // 2

        # Create position encodings for both input and output positions
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, half_dim, 2) * (-math.log(10000.0) / half_dim))

        # Input position encodings
        pe_input = torch.zeros(max_len, half_dim)
        pe_input[:, 0::2] = torch.sin(position * div_term)
        pe_input[:, 1::2] = torch.cos(position * div_term)

        # Output position encodings
        pe_output = torch.zeros(max_len, half_dim)
        pe_output[:, 0::2] = torch.sin(position * div_term)
        pe_output[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe_input', pe_input)
        self.register_buffer('pe_output', pe_output)

    def forward(self, x, input_positions, output_positions):
        batch_size, seq_length, _ = x.shape

        # Create a tensor of zeros with the same shape as the input
        pos_encoding = torch.zeros_like(x)

        # For each item in the batch
        for b in range(batch_size):
            for t in range(seq_length):
                # Get the input and output positions for this token
                input_pos = input_positions[b, t] if input_positions is not None else t
                output_pos = output_positions[b, t] if output_positions is not None else t

                if input_pos < self.pe_input.size(0) and output_pos < self.pe_output.size(0):
                    # Fill the first half with input position encoding
                    pos_encoding[b, t, :self.d_model//2] = self.pe_input[input_pos]
                    # Fill the second half with output position encoding
                    pos_encoding[b, t, self.d_model//2:] = self.pe_output[output_pos]

        return x + pos_encoding

class PerceiverAttention(nn.Module):
    def __init__(self, dim, dim_head=64, heads=8):
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm_media = nn.LayerNorm(dim)
        self.norm_latents = nn.LayerNorm(dim)
        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

    def forward(self, x, latents):
        """
        x: [batch_size, seq_len_x, dim]
        latents: [batch_size, seq_len_l, dim]
        """
        batch_size = x.shape[0]

        x = self.norm_media(x)
        latents = self.norm_latents(latents)

        # Ensure latents has correct batch size
        if latents.size(0) != batch_size:
            latents = latents.expand(batch_size, -1, -1)

        q = self.to_q(latents)
        # Handle rearrange function
        q = q.view(batch_size, -1, self.heads, q.size(-1) // self.heads).transpose(1, 2)
        q = q * self.scale

        # Ensure proper concatenation
        kv_input = torch.cat((x, latents), dim=1)  # concatenate along sequence dimension
        k, v = self.to_kv(kv_input).chunk(2, dim=-1)
        # Handle rearrange function
        k = k.view(batch_size, -1, self.heads, k.size(-1) // self.heads).transpose(1, 2)
        v = v.view(batch_size, -1, self.heads, v.size(-1) // self.heads).transpose(1, 2)

        sim = torch.einsum('b h i d, b h j d -> b h i j', q, k)
        attn = sim.softmax(dim=-1)
        out = torch.einsum('b h i j, b h j d -> b h i d', attn, v)
        # Handle rearrange function
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.heads * out.size(-1))

        return self.to_out(out)

class GatedCrossAttentionBlock(nn.Module):
    def __init__(self, dim, dim_head=64, heads=8, ff_mult=4):
        super().__init__()
        self.attn = PerceiverAttention(dim=dim, dim_head=dim_head, heads=heads)
        self.attn_gate = nn.Parameter(torch.tensor([0.]))
        self.ff = FeedForward(dim, mult=ff_mult)
        self.ff_gate = nn.Parameter(torch.tensor([0.]))

    def forward(self, x, media):
        """
        x: [batch_size, seq_len_x, dim]
        media: [batch_size, seq_len_m, dim]
        """
        batch_size = x.shape[0]
        target_batch_size = media.size(0)

        # Handle batch size mismatch
        if batch_size > target_batch_size:
            media = media.expand(batch_size, -1, -1)
        elif batch_size < target_batch_size:
            x = x.expand(target_batch_size, -1, -1)

        gate = self.attn_gate.tanh()
        x = self.attn(media, x) * gate + x
        x = self.ff(x) * self.ff_gate.tanh() + x
        return x

class PerceiverResampler(nn.Module):
    def __init__(self, dim, depth, dim_head=64, heads=8, num_latents=64):
        super().__init__()
        # Initialize latents without batch dimension
        self.latents = nn.Parameter(torch.randn(num_latents, dim))
        self.layers = nn.ModuleList([])

        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PerceiverAttention(dim=dim, dim_head=dim_head, heads=heads),
                FeedForward(dim=dim)
            ]))

    def forward(self, x):
        batch_size = x.shape[0]
        if len(x.shape) == 2:
          x = x.unsqueeze(1)  # [batch_size, embedding_dim] -> [batch_size, 1, embedding_dim]

        # Expand latents to batch size (handle repeat function)
        latents = self.latents.unsqueeze(0).expand(batch_size, -1, -1)

        for attn, ff in self.layers:
            latents = attn(x, latents) + latents
            latents = ff(latents) + latents

        return latents

class FeedForward(nn.Module):
    def __init__(self, dim, mult=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim * mult, bias=False),
            nn.GELU(),
            nn.Linear(dim * mult, dim, bias=False)
        )

    def forward(self, x):
        return self.net(x)


class SCGPTSmilesGPT(nn.Module):
    def __init__(self, max_len,
                 cross_attn_every=3, dim_head=64, heads=8,
                 perceiver_depth=2, perceiver_num_latents=64,
                 working_dir="./scgpt_smiles_workdir",
                 embedding_dim=512):
        super().__init__()

        # Load molGPT model
        self.smilesgpt_model, self.smilesgpt_tokenizer = load_mol_gpt_model()
        self.max_len = max_len
        self.working_dir = working_dir

        if self.smilesgpt_tokenizer.pad_token is None:
            self.smilesgpt_tokenizer.pad_token = self.smilesgpt_tokenizer.eos_token
            self.smilesgpt_model.config.pad_token_id = self.smilesgpt_model.config.eos_token_id

        self.cross_attn_every = cross_attn_every

        # Add projection layer to adapt scGPT embedding dimension to molGPT dimension if needed
        self.projection = nn.Linear(embedding_dim, self.smilesgpt_model.config.n_embd)

        self.positional_encoding = DoublePositionalEncoding(
            self.smilesgpt_model.config.n_embd,
            max_len=max_len
        )

        self.cell_perceiver = PerceiverResampler(
            dim=self.smilesgpt_model.config.n_embd,
            depth=perceiver_depth,
            dim_head=dim_head,
            heads=heads,
            num_latents=perceiver_num_latents
        )

        num_gpt_layers = len(self.smilesgpt_model.transformer.h)
        self.cross_attn = nn.ModuleList([
            GatedCrossAttentionBlock(
                dim=self.smilesgpt_model.config.n_embd,
                dim_head=dim_head,
                heads=heads
            )
            for _ in range(num_gpt_layers)
        ])

        self.layers = nn.ModuleList()
        for i, block in enumerate(self.smilesgpt_model.transformer.h):
            self.layers.append(block)
            if i % cross_attn_every == 0 and i != 0:
                self.layers.append(
                    GatedCrossAttentionBlock(
                        dim=self.smilesgpt_model.config.n_embd,
                        dim_head=dim_head,
                        heads=heads
                    )
                )

    def forward(self, cell_embeddings, targets=None, optimize=False, order=None):
        device = next(self.parameters()).device

        # Project cell embeddings to match GPT embedding dimension if necessary
        cell_embeddings = self.projection(cell_embeddings)
        # print(f"Cell embeddings shape: {cell_embeddings.shape}")

        processed_cells = self.cell_perceiver(cell_embeddings)

        gpt_input = self.smilesgpt_tokenizer.encode_plus(
            "<s>",  # Use the BOS token specific to SMILES-GPT
            return_tensors="pt",
            padding='max_length',
            max_length=self.max_len,
            truncation=True
        ).to(device)

        input_ids = gpt_input.input_ids.long()
        seq_length = input_ids.size(1)
        batch_size = cell_embeddings.size(0)

        # print("Input IDs shape:", input_ids.shape)
        # print("Batch size:", batch_size)


        hidden_states = self.smilesgpt_model.transformer.wte(input_ids)

        if order is None:
            order = torch.arange(seq_length, device=device).unsqueeze(0).repeat(batch_size, 1)

        if order.size(1) > seq_length:
            order = order[:, :seq_length]
        elif order.size(1) < seq_length:
            padding = torch.arange(order.size(1), seq_length, device=device).unsqueeze(0).repeat(batch_size, 1)
            order = torch.cat([order, padding], dim=1)

        if not optimize:
            reordered_input_ids = torch.zeros_like(input_ids)
            for b in range(batch_size):
                reordered_input_ids[b] = input_ids[b, order[b]]

            hidden_states = self.smilesgpt_model.transformer.wte(reordered_input_ids)

        input_positions = order
        output_positions = torch.roll(order, -1, dims=1)
        output_positions[:, -1] = order[:, 0]

        hidden_states = self.positional_encoding(hidden_states, input_positions, output_positions)

        attention_mask = gpt_input.attention_mask
        num_heads = self.smilesgpt_model.config.n_head

        attention_mask = attention_mask.view(batch_size, 1, 1, seq_length)
        attention_mask = attention_mask.expand(batch_size, num_heads, seq_length, seq_length)
        attention_mask = attention_mask.to(dtype=hidden_states.dtype)

        seq_indices = torch.arange(seq_length, device=device)
        expanded_seq_indices_i = seq_indices.unsqueeze(1).expand(seq_length, seq_length)
        expanded_seq_indices_j = seq_indices.unsqueeze(0).expand(seq_length, seq_length)

        causal_mask = torch.zeros((batch_size, seq_length, seq_length), device=device)
        for b in range(batch_size):
            order_b = order[b]
            order_i = order_b[expanded_seq_indices_i]
            order_j = order_b[expanded_seq_indices_j]
            causal_mask[b] = (order_j <= order_i).float()

        causal_mask = causal_mask.unsqueeze(1)
        combined_mask = attention_mask * causal_mask

        for i, layer in enumerate(self.layers):
            if isinstance(layer, GatedCrossAttentionBlock):
                hidden_states = layer(hidden_states, processed_cells)
            else:
                hidden_states = layer(hidden_states, attention_mask=combined_mask)[0]

        logits = self.smilesgpt_model.lm_head(hidden_states)

        if targets is None:
            if optimize:
                return logits[:, [-1], :], None
            return logits, None

        if targets is not None:
            shuffled_targets = torch.zeros_like(targets)
            for b in range(batch_size):
                shuffled_targets[b] = targets[b, order[b]]

            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                shuffled_targets.view(-1),
                ignore_index=-1
            )
        else:
            loss = None

        return logits, loss

    def generate_smiles(self, cell_embedding, max_length=200, top_p=0.95):
        device = next(self.parameters()).device

        if cell_embedding.dim() == 1:
            cell_embedding = cell_embedding.unsqueeze(0)


        print(f"Cell embeddings shape: {cell_embedding.shape}")

        # Project cell embeddings to match GPT embedding dimension if necessary
        cell_embedding = self.projection(cell_embedding)
        processed_cells = self.cell_perceiver(cell_embedding)

        # Start with BOS token
        input_ids = torch.tensor([[self.smilesgpt_tokenizer.bos_token_id]]).to(device)

        for _ in range(max_length):
            inputs_embeds = self.smilesgpt_model.transformer.wte(input_ids)
            inputs_embeds = self.positional_encoding(inputs_embeds, None, None)

            hidden_states = inputs_embeds

            for i, layer in enumerate(self.layers):
                if isinstance(layer, GatedCrossAttentionBlock):
                    hidden_states = layer(hidden_states, processed_cells)
                else:
                    hidden_states = layer(hidden_states, attention_mask=None)[0]

            next_token_logits = self.smilesgpt_model.lm_head(hidden_states[:, -1, :])

            filtered_logits = top_p_filtering(next_token_logits, top_p=top_p)
            probs = F.softmax(filtered_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)

            input_ids = torch.cat([input_ids, next_token], dim=-1)

            if next_token.item() == self.smilesgpt_tokenizer.eos_token_id:
                break

        return self.smilesgpt_tokenizer.decode(input_ids[0], skip_special_tokens=True)

    def state_dict(self):
        state_dict = super().state_dict()
        state_dict['cell_perceiver'] = self.cell_perceiver.state_dict()
        state_dict['cross_attn'] = self.cross_attn.state_dict()
        return state_dict

    def load_state_dict(self, state_dict):
        cell_perceiver_state = state_dict.pop('cell_perceiver')
        cross_attn_state = state_dict.pop('cross_attn')

        super().load_state_dict(state_dict)

        self.cell_perceiver.load_state_dict(cell_perceiver_state)
        self.cross_attn.load_state_dict(cross_attn_state)


def prepare_all_drugs_dataset(adata_dir, smiles_path, tokenizer, max_length,
                           working_dir, batch_size=25):
    import scanpy as sc
    import pandas as pd

    # Load SMILES data
    smiles_df = pd.read_csv(smiles_path)

    all_pairs = []

    # Get all h5ad files in the directory
    drug_files = [f for f in os.listdir(adata_dir) if f.endswith('.h5ad')]
    print(f"Found {len(drug_files)} drug files in {adata_dir}")

    for drug_file in drug_files:
        drug_path = os.path.join(adata_dir, drug_file)
        try:
            # Extract drug name from filename
            import re
            # Regular expression to extract the drug name
            match = re.search(r'([A-Za-z0-9-]+)_block', drug_file)

            if match:
                drug_name = match.group(1)
            else:
                drug_name = None  # Or handle the case when no match is found
            print(f"Processing {drug_name} from {drug_file}")

            # Skip if drug not in SMILES data
            if not any(smiles_df['drug'] == drug_name):
                print(f"No SMILES data for {drug_name}, skipping")
                continue

            # Load the h5ad file with pre-computed embeddings
            drug_adata = sc.read_h5ad(drug_path)

            # Get the precomputed differential embedding
            diff_embedding = torch.tensor(drug_adata.obsm["X_scGPT"].mean(axis=0))

            # Get SMILES for this drug
            drug_smiles_df = smiles_df[smiles_df['drug'] == drug_name]

            for _, row in drug_smiles_df.iterrows():
                all_pairs.append(
                    SCGPTSmilesPair(
                        cell_embedding=diff_embedding,
                        smiles_string=row['SMILES']
                    )
                )

            print(f"Added {len(drug_smiles_df)} pairs for {drug_name}")

        except Exception as e:
            print(f"Error processing {drug_file}: {e}")
            continue

    print(f"Created a total of {len(all_pairs)} cell-SMILES pairs")
    return all_pairs


def top_p_filtering(logits, top_p=0.9, filter_value=-float('Inf')):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    sorted_indices_to_remove = cumulative_probs > top_p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = filter_value

    return logits


class SCGPTSmilesPair:
    def __init__(self, cell_embedding, smiles_string):
        self.cell_embedding = cell_embedding
        self.smiles_string = smiles_string


class CellSmilesDataset(torch.utils.data.Dataset):
    def __init__(self, pairs, tokenizer, max_length):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]

        smiles_encoding = self.tokenizer(
            pair.smiles_string,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "cell_embedding": pair.cell_embedding,
            "smiles_tokens": smiles_encoding.input_ids.squeeze(0),
            "attention_mask": smiles_encoding.attention_mask.squeeze(0)
        }


def collate_batch(batch):
    cell_embeddings = torch.stack([item["cell_embedding"] for item in batch])
    smiles_tokens = torch.stack([item["smiles_tokens"] for item in batch])
    attention_masks = torch.stack([item["attention_mask"] for item in batch])

    return {
        "cell_embeddings": cell_embeddings,
        "smiles_tokens": smiles_tokens,
        "attention_masks": attention_masks
    }


def train_scgpt_smiles_gpt(model, train_loader, val_loader, num_epochs, device,
                         l2_reg=1e-5, learning_rate=1e-4, checkpoints_dir="./checkpoints"):
    os.makedirs(checkpoints_dir, exist_ok=True)

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=l2_reg)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    best_val_loss = float('inf')
    training_log = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_tokens = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            cell_embeddings = batch["cell_embeddings"].to(device)
            smiles_tokens = batch["smiles_tokens"].to(device)

            optimizer.zero_grad()

            outputs, loss = model(cell_embeddings, targets=smiles_tokens)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()

            predicted_token_ids = torch.argmax(outputs, dim=-1)
            mask = smiles_tokens != model.smilesgpt_tokenizer.pad_token_id
            correct = (predicted_token_ids[mask] == smiles_tokens[mask]).sum().item()

            total = mask.sum().item()
            total_correct += correct
            total_tokens += total

        avg_loss = total_loss / len(train_loader)
        token_accuracy = total_correct / total_tokens * 100
        perplexity = math.exp(avg_loss)

        print(f"\nEpoch {epoch+1}/{num_epochs}, Avg Loss: {avg_loss:.4f}, Perplexity: {perplexity:.4f}")
        print(f"Token Accuracy: {token_accuracy:.2f}%")

        val_loss, val_perplexity, val_accuracy = validate_model(model, val_loader, device)
        print(f"Validation Loss: {val_loss:.4f}, Perplexity: {val_perplexity:.4f}, Accuracy: {val_accuracy:.2f}%")

        training_log.append({
            'epoch': epoch+1,
            'train_loss': avg_loss,
            'train_perplexity': perplexity,
            'train_accuracy': token_accuracy,
            'val_loss': val_loss,
            'val_perplexity': val_perplexity,
            'val_accuracy': val_accuracy
        })

        checkpoint_path = os.path.join(checkpoints_dir, f"scgptsmilesgpt_epoch_{epoch+1}.pth")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path} (Validation loss improved)")

        scheduler.step()

    loss_df = pd.DataFrame(training_log)
    loss_df.to_csv(os.path.join(checkpoints_dir, "training_log.csv"), index=False)

    return loss_df


def validate_model(model, val_loader, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_tokens = 0

    with torch.no_grad():
        for batch in val_loader:
            cell_embeddings = batch["cell_embeddings"].to(device)
            smiles_tokens = batch["smiles_tokens"].to(device)

            outputs, loss = model(cell_embeddings, targets=smiles_tokens)

            total_loss += loss.item()

            predicted_token_ids = torch.argmax(outputs, dim=-1)
            mask = smiles_tokens != model.smilesgpt_tokenizer.pad_token_id
            correct = (predicted_token_ids[mask] == smiles_tokens[mask]).sum().item()

            total = mask.sum().item()
            total_correct += correct
            total_tokens += total

    avg_loss = total_loss / len(val_loader)
    token_accuracy = total_correct / total_tokens * 100
    perplexity = math.exp(avg_loss)

    return avg_loss, perplexity, token_accuracy


def generate_and_evaluate(model, test_data, metrics_funcs=None, device=None):
    if device is None:
        device = next(model.parameters()).device

    model.eval()
    results = []

    for pair in tqdm(test_data, desc="Generating SMILES"):
        cell_embedding = pair.cell_embedding.to(device)
        ground_truth = pair.smiles_string

        generated_smiles = model.generate_smiles(cell_embedding)

        result = {
            "ground_truth": ground_truth,
            "generated": generated_smiles
        }

        if metrics_funcs:
            for metric_name, metric_func in metrics_funcs.items():
                result[metric_name] = metric_func(ground_truth, generated_smiles)

        results.append(result)

    if metrics_funcs:
        aggregate_results = {}
        for metric_name in metrics_funcs.keys():
            values = [r[metric_name] for r in results if metric_name in r]
            aggregate_results[f"avg_{metric_name}"] = sum(values) / len(values)

        print("Evaluation Results:")
        for metric_name, value in aggregate_results.items():
            print(f"{metric_name}: {value:.4f}")

    return results


def visualize_training(log_df, save_path=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(log_df['epoch'], log_df['train_loss'], label='Training')
    ax1.plot(log_df['epoch'], log_df['val_loss'], label='Validation')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss vs. Epoch')
    ax1.legend()

    ax2.plot(log_df['epoch'], log_df['train_accuracy'], label='Training')
    ax2.plot(log_df['epoch'], log_df['val_accuracy'], label='Validation')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title('Token Accuracy vs. Epoch')
    ax2.legend()

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

Overwriting scgpt_smiles_model.py


In [18]:
%%writefile scgpt_smiles_main.py

import torch
import torch.nn as nn
import argparse
import os
import pandas as pd
import scanpy as sc
from torch.utils.data import DataLoader
import sys

# Make sure the paths are available
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from scgpt_smiles_model import (
    DoublePositionalEncoding, PerceiverResampler,
    GatedCrossAttentionBlock, top_p_filtering, SCGPTSmilesPair,
    CellSmilesDataset, collate_batch, train_scgpt_smiles_gpt,
    generate_and_evaluate, visualize_training, SCGPTSmilesGPT,
    prepare_all_drugs_dataset
)

from molgpt_loader import load_mol_gpt_model


def split_dataset_by_morgan_fingerprint(pairs, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    from rdkit import Chem
    from rdkit.Chem import AllChem
    import numpy as np
    from sklearn.cluster import KMeans
    from sklearn.metrics.pairwise import euclidean_distances
    import random

    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-10

    np.random.seed(seed)
    random.seed(seed)

    smiles_list = [pair.smiles_string for pair in pairs]

    fingerprints = []
    valid_indices = []

    for i, smiles in enumerate(smiles_list):
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
                fingerprints.append(np.array(fp))
                valid_indices.append(i)
        except Exception as e:
            print(f"Error processing SMILES {smiles}: {e}")

    valid_pairs = [pairs[i] for i in valid_indices]
    X = np.array(fingerprints)

    if len(valid_pairs) == 0:
        raise ValueError("No valid molecules found in the dataset")

    n_clusters = min(40, len(X))
    kmeans = KMeans(n_clusters=n_clusters, random_state=seed)
    cluster_labels = kmeans.fit_predict(X)

    cluster_centers = kmeans.cluster_centers_

    cluster_indices = {cluster_id: [] for cluster_id in range(n_clusters)}
    for idx, cluster_id in enumerate(cluster_labels):
        cluster_indices[cluster_id].append(idx)

    unique_clusters = list(range(n_clusters))

    def split_by_cluster_dissimilarity(unique_clusters, cluster_centers, train_ratio, val_ratio):
        n_clusters = len(unique_clusters)
        n_train = int(n_clusters * train_ratio)
        n_val = int(n_clusters * val_ratio)

        distances = euclidean_distances(cluster_centers)

        current_cluster = np.random.choice(unique_clusters)
        train_clusters = [current_cluster]
        remaining_clusters = set(unique_clusters) - {current_cluster}

        while len(train_clusters) < n_train:
            avg_distances = []
            for cluster in remaining_clusters:
                avg_dist = np.mean([distances[cluster, train_cluster] for train_cluster in train_clusters])
                avg_distances.append((cluster, avg_dist))

            next_cluster = max(avg_distances, key=lambda x: x[1])[0]
            train_clusters.append(next_cluster)
            remaining_clusters.remove(next_cluster)

        current_cluster = np.random.choice(list(remaining_clusters))
        val_clusters = [current_cluster]
        remaining_clusters = remaining_clusters - {current_cluster}

        while len(val_clusters) < n_val:
            avg_distances = []
            for cluster in remaining_clusters:
                avg_dist = np.mean([distances[cluster, val_cluster] for val_cluster in val_clusters])
                avg_distances.append((cluster, avg_dist))

            next_cluster = max(avg_distances, key=lambda x: x[1])[0]
            val_clusters.append(next_cluster)
            remaining_clusters.remove(next_cluster)

        test_clusters = list(remaining_clusters)

        return train_clusters, val_clusters, test_clusters

    train_clusters, val_clusters, test_clusters = split_by_cluster_dissimilarity(
        unique_clusters, cluster_centers, train_ratio, val_ratio
    )

    train_indices = [valid_indices[idx] for cluster in train_clusters for idx in cluster_indices[cluster]]
    val_indices = [valid_indices[idx] for cluster in val_clusters for idx in cluster_indices[cluster]]
    test_indices = [valid_indices[idx] for cluster in test_clusters for idx in cluster_indices[cluster]]

    train_pairs = [pairs[i] for i in train_indices]
    val_pairs = [pairs[i] for i in val_indices]
    test_pairs = [pairs[i] for i in test_indices]

    print(f"Dataset split: {len(train_pairs)} training, {len(val_pairs)} validation, {len(test_pairs)} test samples")
    print(f"Split by clusters: {len(train_clusters)} training, {len(val_clusters)} validation, {len(test_clusters)} test clusters")

    return train_pairs, val_pairs, test_pairs


def main():
    parser = argparse.ArgumentParser(description="Train and evaluate scGPT to SMILES model")
    parser.add_argument("--adata_dir", type=str, required=True, help="Directory containing anndata files with scGPT embeddings")
    parser.add_argument("--smiles_path", type=str, required=True, help="Path to SMILES CSV file")
    parser.add_argument("--reprocess_data", action="store_true", help="Force reprocessing of data even if cached files exist")
    parser.add_argument("--working_dir", type=str, default="./scgpt_smiles_workdir", help="Working directory")
    parser.add_argument("--batch_size", type=int, default=16, help="Batch size for training")
    parser.add_argument("--max_length", type=int, default=512, help="Maximum sequence length")
    parser.add_argument("--num_epochs", type=int, default=20, help="Number of training epochs")
    parser.add_argument("--learning_rate", type=float, default=5e-5, help="Learning rate")
    parser.add_argument("--l2_reg", type=float, default=1e-5, help="L2 regularization")
    parser.add_argument("--cross_attn_every", type=int, default=3, help="Cross attention frequency")
    parser.add_argument("--perceiver_depth", type=int, default=2, help="Perceiver depth")
    parser.add_argument("--seed", type=int, default=42, help="Random seed")
    parser.add_argument("--checkpoints_dir", type=str, default="./checkpoints", help="Checkpoints directory")
    parser.add_argument("--eval_only", action="store_true", help="Run evaluation only")
    parser.add_argument("--checkpoint_path", type=str, help="Path to checkpoint for evaluation")
    parser.add_argument("--train_ratio", type=float, default=0.7, help="Ratio of training data")
    parser.add_argument("--val_ratio", type=float, default=0.15, help="Ratio of validation data")
    parser.add_argument("--test_ratio", type=float, default=0.15, help="Ratio of test data")
    parser.add_argument("--embedding_dim", type=int, default=512, help="Dimension of scGPT embeddings")

    args = parser.parse_args()

    torch.manual_seed(args.seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load molGPT tokenizer
    _, tokenizer = load_mol_gpt_model()

    # Create the model
    model = SCGPTSmilesGPT(
        max_len=args.max_length,
        cross_attn_every=args.cross_attn_every,
        perceiver_depth=args.perceiver_depth,
        working_dir=args.working_dir,
        embedding_dim=args.embedding_dim
    ).to(device)

    import pickle
    import os

    # File paths for saving processed data
    pairs_file = os.path.join(args.working_dir, "processed_pairs.pkl")
    train_pairs_file = os.path.join(args.working_dir, "train_pairs.pkl")
    val_pairs_file = os.path.join(args.working_dir, "val_pairs.pkl")
    test_pairs_file = os.path.join(args.working_dir, "test_pairs.pkl")

    # Create working directory if it doesn't exist
    os.makedirs(args.working_dir, exist_ok=True)

    print("Checking for cached data...")
    if os.path.exists(pairs_file) and not args.reprocess_data:
        # Load previously processed pairs
        print("Loading cached pairs...")
        with open(pairs_file, 'rb') as f:
            pairs = pickle.load(f)

        if os.path.exists(train_pairs_file) and os.path.exists(val_pairs_file) and os.path.exists(test_pairs_file):
            print("Loading cached split datasets...")
            with open(train_pairs_file, 'rb') as f:
                train_pairs = pickle.load(f)
            with open(val_pairs_file, 'rb') as f:
                val_pairs = pickle.load(f)
            with open(test_pairs_file, 'rb') as f:
                test_pairs = pickle.load(f)
        else:
            # Split the loaded pairs
            print(f"Splitting dataset with train_ratio={args.train_ratio}, val_ratio={args.val_ratio}, test_ratio={args.test_ratio}")
            train_pairs, val_pairs, test_pairs = split_dataset_by_morgan_fingerprint(
                pairs,
                train_ratio=args.train_ratio,
                val_ratio=args.val_ratio,
                test_ratio=args.test_ratio,
                seed=args.seed
            )

            # Save the split
            with open(train_pairs_file, 'wb') as f:
                pickle.dump(train_pairs, f)
            with open(val_pairs_file, 'wb') as f:
                pickle.dump(val_pairs, f)
            with open(test_pairs_file, 'wb') as f:
                pickle.dump(test_pairs, f)
    else:
        # Process data from scratch
        print("Processing dataset...")
        pairs = prepare_all_drugs_dataset(
            adata_dir=args.adata_dir,
            smiles_path=args.smiles_path,
            tokenizer=tokenizer,
            max_length=args.max_length,
            working_dir=args.working_dir,
            batch_size=args.batch_size
        )

        # Save processed pairs
        with open(pairs_file, 'wb') as f:
            pickle.dump(pairs, f)

        if len(pairs) == 0:
            print("No valid cell-SMILES pairs found. Check your input data.")
            return

        # Split dataset
        print(f"Splitting dataset with train_ratio={args.train_ratio}, val_ratio={args.val_ratio}, test_ratio={args.test_ratio}")
        train_pairs, val_pairs, test_pairs = split_dataset_by_morgan_fingerprint(
            pairs,
            train_ratio=args.train_ratio,
            val_ratio=args.val_ratio,
            test_ratio=args.test_ratio,
            seed=args.seed
        )

        # Save the split
        with open(train_pairs_file, 'wb') as f:
            pickle.dump(train_pairs, f)
        with open(val_pairs_file, 'wb') as f:
            pickle.dump(val_pairs, f)
        with open(test_pairs_file, 'wb') as f:
            pickle.dump(test_pairs, f)

    # Create datasets and dataloaders
    train_dataset = CellSmilesDataset(train_pairs, tokenizer, args.max_length)
    val_dataset = CellSmilesDataset(val_pairs, tokenizer, args.max_length)
    test_dataset = CellSmilesDataset(test_pairs, tokenizer, args.max_length)

    train_loader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        collate_fn=collate_batch,
        num_workers=4
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=collate_batch,
        num_workers=4
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=collate_batch,
        num_workers=4
    )

    if args.eval_only and args.checkpoint_path:
        print(f"Loading checkpoint from {args.checkpoint_path}")
        model.load_state_dict(torch.load(args.checkpoint_path, map_location=device))

        from rdkit import Chem
        from rdkit.Chem import AllChem, DataStructs

        def calculate_tanimoto_similarity(smiles1, smiles2):
            mol1 = Chem.MolFromSmiles(smiles1)
            mol2 = Chem.MolFromSmiles(smiles2)

            if mol1 is None or mol2 is None:
                return 0.0

            fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=1024)
            fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=1024)

            return DataStructs.TanimotoSimilarity(fp1, fp2)

        def valid_molecule_ratio(smiles1, smiles2):
            mol = Chem.MolFromSmiles(smiles1)
            return 1.0 if mol is not None else 0.0

        metrics_funcs = {
            "tanimoto": calculate_tanimoto_similarity,
            "valid": valid_molecule_ratio
        }

        print("Generating and evaluating SMILES...")
        results = generate_and_evaluate(model, test_pairs, metrics_funcs, device)

        results_df = pd.DataFrame(results)
        results_path = os.path.join(args.checkpoints_dir, "evaluation_results.csv")
        results_df.to_csv(results_path, index=False)
        print(f"Evaluation results saved to {results_path}")
    else:
        print(f"Training for {args.num_epochs} epochs")
        log_df = train_scgpt_smiles_gpt(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            num_epochs=args.num_epochs,
            device=device,
            l2_reg=args.l2_reg,
            learning_rate=args.learning_rate,
            checkpoints_dir=args.checkpoints_dir
        )

        visualize_training(log_df, save_path=os.path.join(args.checkpoints_dir, "training_curves.png"))


if __name__ == "__main__":
    main()

Overwriting scgpt_smiles_main.py


## run lines

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
# os.chdir("/content/drive/MyDrive/Colab Notebooks/esm cell state/")
os.chdir("/content/")

In [10]:
!ls "/content/"

drive  molgpt_loader.py  sample_data  scgpt_smiles_main.py  scgpt_smiles_model.py


In [11]:
import pandas as pd
pd.read_csv("/content/drive/MyDrive/Colab Notebooks/esm cell state/smiles_df.csv")

,drug,SMILES
0,Dabrafenib,CC(C)(C)C1=NC(=C(S1)C2=NC(=NC=C2)N)C3=C(C(=CC=...
1,Encorafenib,C[C@@H](CNC1=NC=CC(=N1)C2=CN(N=C2C3=C(C(=CC(=C...
2,Paclitaxel,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@...
3,9-ING-41,CN1C=C(C2=CC3=C(C=C21)OCO3)C4=C(C(=O)NC4=O)C5=...
4,Sapanisertib,CC(C)N1C2=NC=NC(=C2C(=N1)C3=CC4=C(C=C3)OC(=N4)N)N
...,...,...
88,Torkinib,CC(C)N1C2=NC=NC(=C2C(=N1)C3=CC4=C(N3)C=CC(=C4)O)N
89,AZD2858,CN1CCN(CC1)S(=O)(=O)C2=CC=C(C=C2)C3=CN=C(C(=N3...
90,Bortezomib,B([C@H](CC(C)C)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)...
91,LY-2584702 (tosylate salt),CC1=CC=C(C=C1)S(=O)(=O)O.CN1C=C(N=C1C2CCN(CC2)...


In [ ]:
!python scgpt_smiles_main.py --adata_dir "/content/drive/MyDrive/Colab Notebooks/esm cell state/difference counts/" \
--smiles_path "/content/drive/MyDrive/Colab Notebooks/esm cell state/smiles_df.csv" \
--working_dir "/content/drive/MyDrive/Colab Notebooks/esm cell state/scgpt_workdir" \
--batch_size 1 --max_length 256 --num_epochs 10 --learning_rate 1e-5 \
--checkpoints_dir "/content/drive/MyDrive/Colab Notebooks/esm cell state/checkpoints"

2025-03-16 01:18:41.729627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742087921.750890    9670 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742087921.757046    9670 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 01:18:41.778726: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using device: cuda
Checking for cached data...
Loading cached pairs...
Loading cached split datasets...
Training for 10 epoch

In [3]:
!ls  "/content/drive/MyDrive/Colab Notebooks/esm cell state/scgpt_workdir"

processed_pairs.pkl


In [15]:
import pickle

file_path = "/content/drive/MyDrive/Colab Notebooks/esm cell state/scgpt_workdir/processed_pairs.pkl"

with open(file_path, "rb") as file:
    data = pickle.load(file)

# Display the structure of the data
type(data), len(data) if hasattr(data, '__len__') else None

sample_size = 5 if len(data) > 5 else len(data)
sample_items = data[:sample_size]

data_types = {i: type(item) for i, item in enumerate(sample_items)}

unique_types = set(type(item) for item in data)

print(f"Total items: {len(data)}")
print(f"Sample items: {sample_items}")
print(f"Data types in sample: {data_types}")
print(f"Unique types: {unique_types}")

Total items: 27982
Sample items: [<scgpt_smiles_model.SCGPTSmilesPair object at 0x7c3d0bb2a4d0>, <scgpt_smiles_model.SCGPTSmilesPair object at 0x7c3d0bb9b1d0>, <scgpt_smiles_model.SCGPTSmilesPair object at 0x7c3d0bb9b850>, <scgpt_smiles_model.SCGPTSmilesPair object at 0x7c3d0bb9b990>, <scgpt_smiles_model.SCGPTSmilesPair object at 0x7c3d0bb9aa50>]
Data types in sample: {0: <class 'scgpt_smiles_model.SCGPTSmilesPair'>, 1: <class 'scgpt_smiles_model.SCGPTSmilesPair'>, 2: <class 'scgpt_smiles_model.SCGPTSmilesPair'>, 3: <class 'scgpt_smiles_model.SCGPTSmilesPair'>, 4: <class 'scgpt_smiles_model.SCGPTSmilesPair'>}
Unique types: {<class 'scgpt_smiles_model.SCGPTSmilesPair'>}


In [16]:
sample_item = data[0]

attributes = dir(sample_item)
filtered_attributes = [attr for attr in attributes if not attr.startswith("__")]

attribute_values = {attr: getattr(sample_item, attr) for attr in filtered_attributes if hasattr(sample_item, attr)}

print(f"Attributes: {filtered_attributes}")
print(f"Attribute Values: {attribute_values}")


Attributes: ['cell_embedding', 'smiles_string']
Attribute Values: {'cell_embedding': tensor([ 6.6132e-03, -3.0450e-03, -1.3491e-03,  1.6596e-03,  4.5485e-03,
         1.0073e-03,  1.4558e-03, -1.5186e-04,  3.6667e-04,  3.9284e-03,
         2.1864e-03,  7.1017e-03,  2.8821e-03, -3.1636e-03, -4.0230e-04,
         1.6058e-03,  1.5317e-03,  8.1104e-03, -1.1847e-03,  7.2079e-04,
         2.9614e-03,  5.9930e-03, -6.8728e-04, -1.3140e-03,  3.7801e-03,
         1.1000e-03,  1.1637e-03,  5.2239e-03,  6.1034e-03, -1.3406e-03,
        -5.2022e-03, -6.4475e-03, -2.2136e-03,  4.5650e-04, -4.4613e-03,
         1.0329e-03,  3.2738e-03, -2.3091e-03, -4.4869e-03,  7.6485e-04,
         5.5050e-04,  1.4897e-03,  1.0159e-03, -5.5807e-03, -1.3702e-03,
        -3.7904e-03, -3.7659e-04, -1.1254e-03,  3.9252e-03, -3.4051e-03,
        -2.1338e-04,  2.0985e-03,  1.6477e-03,  6.3730e-04,  3.1382e-03,
        -4.1852e-03, -6.0325e-03, -7.0772e-04,  2.3096e-03,  4.0186e-03,
        -1.1126e-03, -5.3812e-03, -8.62

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import anndata
import scanpy as sc
import os
import numpy as np
from typing import Dict, List, Optional, Tuple, Union
import math
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

from molgpt_loader import load_mol_gpt_model
# Extracting first 5 samples for quick inspection
sample_size = 5
samples = [
    {
        "smiles": pair.smiles_string,
        "embedding_shape": pair.cell_embedding.shape if isinstance(pair.cell_embedding, torch.Tensor) else None
    }
    for pair in data[:sample_size]
]

# Get summary statistics of tensor embeddings
embedding_shapes = [pair.cell_embedding.shape for pair in data if isinstance(pair.cell_embedding, torch.Tensor)]
unique_shapes = set(embedding_shapes)

print(f"Total pairs: {len(data)}")
print(f"Unique embedding shapes: {unique_shapes}")
print(f"Sample entries: {samples}")

Total pairs: 27982
Unique embedding shapes: {torch.Size([512])}
Sample entries: [{'smiles': 'CCN1CCN(CC1)C2=CC=C(C=C2)NC3=CC(=NC=N3)N(C)C(=O)NC4=C(C(=CC(=C4Cl)OC)OC)Cl', 'embedding_shape': torch.Size([512])}, {'smiles': 'CCN1CCN(CC1)C2=CC=C(C=C2)NC3=CC(=NC=N3)N(C)C(=O)NC4=C(C(=CC(=C4Cl)OC)OC)Cl', 'embedding_shape': torch.Size([512])}, {'smiles': 'CCN1CCN(CC1)C2=CC=C(C=C2)NC3=CC(=NC=N3)N(C)C(=O)NC4=C(C(=CC(=C4Cl)OC)OC)Cl', 'embedding_shape': torch.Size([512])}, {'smiles': 'CCN1CCN(CC1)C2=CC=C(C=C2)NC3=CC(=NC=N3)N(C)C(=O)NC4=C(C(=CC(=C4Cl)OC)OC)Cl', 'embedding_shape': torch.Size([512])}, {'smiles': 'CCN1CCN(CC1)C2=CC=C(C=C2)NC3=CC(=NC=N3)N(C)C(=O)NC4=C(C(=CC(=C4Cl)OC)OC)Cl', 'embedding_shape': torch.Size([512])}]


# arc dataset...

## sampling arc-dataset plate 3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/esm cell state/")

In [9]:
!ls "/content/drive/MyDrive/Colab Notebooks/esm cell state/"

 33l_8ep_1024t_1280.torch	 avg_counts_control	    h5ad		    smiles-gpt
 arc_flamingo.ipynb		 avg_counts_filtered	    scgpt_anndata.h5ad	    uce_workdir
'arc virtual cell atlas.ipynb'	 cell			    scGPT_human
 avg_counts			 chemberta_embeddings.pkl   smiles_df.csv
 avg_counts_2			'gb check for data.ipynb'   smiles_embeddings.pkl


In [8]:
import scanpy as sc

In [11]:
adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/scgpt_anndata.h5ad")

In [18]:
X_scGPT = adata.obsm['X_scGPT']

In [22]:
print(X_scGPT.shape)

(470494, 512)


In [30]:
# !rm -rf "/content/drive/MyDrive/Colab Notebooks/esm cell state/difference counts"

In [31]:
import scanpy as sc
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

unique_drugs = adata.obs['drug'].unique()
print(f"Number of unique drugs: {len(unique_drugs)}")
print(unique_drugs)

block_size = 15  # Each block will contain 15 samples
cells_per_drug = {}
for drug in unique_drugs:
    cells_per_drug[drug] = sum(adata.obs['drug'] == drug)
    print(f"Drug {drug}: {cells_per_drug[drug]} cells")

print("Calculating control averages...")
control_data = adata[adata.obs['drug'] == 'DMSO_TF']
# Calculate average for adata.X
control_avg_X = control_data.X.mean(axis=0)
# Calculate average for adata.obsm['X_scGPT']
control_avg_scGPT = control_data.obsm['X_scGPT'].mean(axis=0)

output_dir = "/content/drive/MyDrive/Colab Notebooks/esm cell state/difference counts"
os.makedirs(output_dir, exist_ok=True)

# Calculate how many blocks we'll create
total_blocks = sum([cells_per_drug[drug] // block_size + (1 if cells_per_drug[drug] % block_size > 0 else 0)
                   for drug in unique_drugs if drug != 'DMSO_TF'])
print(f"Total blocks to create: approximately {total_blocks}")

block_counter = 0
for drug in tqdm(unique_drugs, desc="Processing drugs"):
    if drug == 'DMSO_TF':
        continue

    drug_data = adata[adata.obs['drug'] == drug]
    n_cells = drug_data.shape[0]

    # Create blocks of 15 samples each
    n_blocks = n_cells // block_size + (1 if n_cells % block_size > 0 else 0)

    for block_idx in range(n_blocks):
        start_idx = block_idx * block_size
        end_idx = min(start_idx + block_size, n_cells)

        # Get indices for this block
        block_indices = list(range(start_idx, end_idx))

        block_data = drug_data[block_indices].copy()

        # Calculate difference for adata.X
        block_avg_X = block_data.X.mean(axis=0)
        diff_values_X = block_avg_X - control_avg_X
        block_data.X = diff_values_X.reshape(1, -1).repeat(len(block_indices), axis=0)

        # Calculate difference for adata.obsm['X_scGPT']
        if hasattr(block_data, 'obsm') and 'X_scGPT' in block_data.obsm:
            block_avg_scGPT = block_data.obsm['X_scGPT'].mean(axis=0)
            diff_values_scGPT = block_avg_scGPT - control_avg_scGPT

            # Replace original values with differences
            block_data.obsm['X_scGPT'] = diff_values_scGPT.reshape(1, -1).repeat(len(block_indices), axis=0)

        block_data.write_h5ad(f"{output_dir}/difference_{drug}_block{block_idx+1}.h5ad")

        block_counter += 1
        if block_counter % 10 == 0:
            print(f"Completed {block_counter}/{total_blocks} blocks")

print(f"Processing complete. Created {block_counter} difference files.")

Number of unique drugs: 93
['Infigratinib', 'Erdafitinib ', 'Everolimus', 'Pemigatinib', 'Abemaciclib', ..., 'Oleic acid', 'Minodronic acid', 'Anastrozole', 'Adagrasib', 'DMSO_TF']
Length: 93
Categories (93, object): ['4EGI-1', '9-ING-41', 'APTO-253', 'AT7519', ..., 'olaparib',
                          'palbociclib', 'venetoclax', 'vincristine']
Drug Infigratinib: 5167 cells
Drug Erdafitinib : 5118 cells
Drug Everolimus: 11703 cells
Drug Pemigatinib: 4376 cells
Drug Abemaciclib: 4616 cells
Drug Capivasertib: 3121 cells
Drug Gemcitabine: 7448 cells
Drug Irinotecan: 7988 cells
Drug Oxaliplatin: 4693 cells
Drug Alpelisib: 3652 cells
Drug Lapatinib ditosylate: 7903 cells
Drug palbociclib: 3887 cells
Drug Lonafarnib: 1144 cells
Drug Volasertib: 2556 cells
Drug c-Kit-IN-1: 3283 cells
Drug venetoclax: 3334 cells
Drug Capmatinib: 2998 cells
Drug MK-8353: 3557 cells
Drug Regorafenib: 4533 cells
Drug Tucatinib: 8439 cells
Drug Vemurafenib: 7289 cells
Drug BI-3406: 6913 cells
Drug AZD-8055: 6123

Processing drugs:   0%|          | 0/93 [00:00<?, ?it/s]

Completed 10/30807 blocks
Completed 20/30807 blocks
Completed 30/30807 blocks
Completed 40/30807 blocks
Completed 50/30807 blocks
Completed 60/30807 blocks
Completed 70/30807 blocks
Completed 80/30807 blocks
Completed 90/30807 blocks
Completed 100/30807 blocks
Completed 110/30807 blocks
Completed 120/30807 blocks
Completed 130/30807 blocks
Completed 140/30807 blocks
Completed 150/30807 blocks
Completed 160/30807 blocks
Completed 170/30807 blocks
Completed 180/30807 blocks
Completed 190/30807 blocks
Completed 200/30807 blocks
Completed 210/30807 blocks
Completed 220/30807 blocks
Completed 230/30807 blocks
Completed 240/30807 blocks
Completed 250/30807 blocks
Completed 260/30807 blocks
Completed 270/30807 blocks
Completed 280/30807 blocks
Completed 290/30807 blocks
Completed 300/30807 blocks
Completed 310/30807 blocks
Completed 320/30807 blocks
Completed 330/30807 blocks
Completed 340/30807 blocks


Processing drugs:   1%|          | 1/93 [00:21<32:16, 21.04s/it]

Completed 350/30807 blocks
Completed 360/30807 blocks
Completed 370/30807 blocks
Completed 380/30807 blocks
Completed 390/30807 blocks
Completed 400/30807 blocks
Completed 410/30807 blocks
Completed 420/30807 blocks
Completed 430/30807 blocks
Completed 440/30807 blocks
Completed 450/30807 blocks
Completed 460/30807 blocks
Completed 470/30807 blocks
Completed 480/30807 blocks
Completed 490/30807 blocks
Completed 500/30807 blocks
Completed 510/30807 blocks
Completed 520/30807 blocks
Completed 530/30807 blocks
Completed 540/30807 blocks
Completed 550/30807 blocks
Completed 560/30807 blocks
Completed 570/30807 blocks
Completed 580/30807 blocks
Completed 590/30807 blocks
Completed 600/30807 blocks
Completed 610/30807 blocks
Completed 620/30807 blocks
Completed 630/30807 blocks
Completed 640/30807 blocks
Completed 650/30807 blocks
Completed 660/30807 blocks
Completed 670/30807 blocks
Completed 680/30807 blocks


...

Completed 30290/30807 blocks
Completed 30300/30807 blocks
Completed 30310/30807 blocks
Completed 30320/30807 blocks
Completed 30330/30807 blocks
Completed 30340/30807 blocks
Completed 30350/30807 blocks
Completed 30360/30807 blocks
Completed 30370/30807 blocks
Completed 30380/30807 blocks
Completed 30390/30807 blocks
Completed 30400/30807 blocks
Completed 30410/30807 blocks
Completed 30420/30807 blocks
Completed 30430/30807 blocks
Completed 30440/30807 blocks
Completed 30450/30807 blocks
Completed 30460/30807 blocks
Completed 30470/30807 blocks
Completed 30480/30807 blocks
Completed 30490/30807 blocks
Completed 30500/30807 blocks
Completed 30510/30807 blocks
Completed 30520/30807 blocks
Completed 30530/30807 blocks
Completed 30540/30807 blocks
Completed 30550/30807 blocks
Completed 30560/30807 blocks
Completed 30570/30807 blocks
Completed 30580/30807 blocks
Completed 30590/30807 blocks
Completed 30600/30807 blocks


Processing drugs:  98%|█████████▊| 91/93 [31:40<00:37, 18.62s/it]

Completed 30610/30807 blocks
Completed 30620/30807 blocks
Completed 30630/30807 blocks
Completed 30640/30807 blocks
Completed 30650/30807 blocks
Completed 30660/30807 blocks
Completed 30670/30807 blocks
Completed 30680/30807 blocks
Completed 30690/30807 blocks
Completed 30700/30807 blocks
Completed 30710/30807 blocks
Completed 30720/30807 blocks
Completed 30730/30807 blocks
Completed 30740/30807 blocks
Completed 30750/30807 blocks
Completed 30760/30807 blocks
Completed 30770/30807 blocks
Completed 30780/30807 blocks
Completed 30790/30807 blocks
Completed 30800/30807 blocks


Processing drugs: 100%|██████████| 93/93 [31:52<00:00, 20.57s/it]

Processing complete. Created 30807 difference files.


In [32]:
import os

output_dir = "/content/drive/MyDrive/Colab Notebooks/esm cell state/difference counts"
files = os.listdir(output_dir)
print(f"Total number of files in output directory: {len(files)}")

# Optional: count files per drug
drug_counts = {}
for file in files:
    if file.startswith("difference_"):
        drug_name = file.split("_")[1]
        if "block" in drug_name:
            # Handle case where drug name might contain underscore
            parts = file.split("_")
            block_part = [p for p in parts if "block" in p][0]
            drug_name = "_".join(parts[1:parts.index(block_part)])
        else:
            # Handle case where block is separated by underscore
            drug_name = "_".join(file.split("_")[1:-1])

        drug_counts[drug_name] = drug_counts.get(drug_name, 0) + 1

print("\nFiles per drug:")
for drug, count in sorted(drug_counts.items()):
    print(f"{drug}: {count} files")

Total number of files in output directory: 30807

Files per drug:
4EGI-1: 210 files
9-ING-41: 675 files
APTO-253: 311 files
AT7519: 363 files
AZD-7648: 198 files
AZD-8055: 409 files
AZD1390: 275 files
AZD2858: 126 files
Abemaciclib: 308 files
Adagrasib: 201 files
Afatinib: 540 files
Alpelisib: 244 files
Anastrozole: 320 files
BAY1125976: 645 files
BI-3406: 461 files
BI-78D3: 496 files
Belumosudil (mesylate): 220 files
Belzutifan: 188 files
Bentamapimod: 399 files
Bimiralisib: 661 files
Binimetinib: 194 files
Bortezomib: 105 files
Brivudine: 551 files
CP21R7: 206 files
Capivasertib: 209 files
Capmatinib: 200 files
Celecoxib: 270 files
DT-061: 198 files
DTP3: 233 files
Dabrafenib: 828 files
ERK5-IN-2: 629 files
ETC-206: 303 files
EX229: 253 files
Elimusertib hydrochloride: 138 files
Encorafenib: 782 files
Erdafitinib : 342 files
Everolimus: 781 files
Futibatinib: 184 files
GSK1059615: 552 files
Gemcitabine: 497 files
HI-TOPK-032: 197 files
Hydroxyfasudil: 163 files
IQ 1: 194 files
Infigr

# drug embeddings

## mol-gpt

In [7]:
%%writefile molgpt_loader.py
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def load_mol_gpt_model():
    """
    Load the molGPT model from Hugging Face.
    """
    model_name = "msb-roshan/molgpt"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Make sure we have a padding token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    return model, tokenizer

if __name__ == "__main__":
    # Test the model loading
    print("Loading molGPT model...")
    model, tokenizer = load_mol_gpt_model()

    # Print model and tokenizer info
    print(f"Model type: {type(model)}")
    print(f"Model config: {model.config}")
    print(f"Tokenizer vocab size: {tokenizer.vocab_size}")
    print(f"Special tokens: {tokenizer.all_special_tokens}")

    # Test generating a molecule
    print("\nGenerating a molecule...")
    input_text = "C"  # Start with carbon atom
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=50,
            num_return_sequences=1,
            do_sample=True,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode and print
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Generated SMILES: {generated_text}")

    # Try to import RDKit for validation
    try:
        from rdkit import Chem
        mol = Chem.MolFromSmiles(generated_text)
        if mol:
            print("Generated a valid molecule!")
        else:
            print("Generated string is not a valid SMILES.")
    except ImportError:
        print("RDKit not available for validation.")

Writing molgpt_loader.py


# cell embeddings

## uce torch model load test

In [69]:
import torch
model = torch.load("/content/drive/MyDrive/Colab Notebooks/esm cell state/33l_8ep_1024t_1280.torch", map_location="cpu")
print(model)


<ipython-input-69-0361e4380a6b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/content/drive/MyDrive/Colab Notebooks/esm cell state/33l_8ep_1024t_1280

OrderedDict([('pos_encoder.pe', tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00]],

        [[ 8.4147e-01,  5.4030e-01,  8.3367e-01,  ...,  1.0000e+00,
           1.0145e-04,  1.0000e+00]],

        [[ 9.0930e-01, -4.1615e-01,  9.2082e-01,  ...,  1.0000e+00,
           2.0290e-04,  1.0000e+00]],

        ...,

        [[-9.7062e-02,  9.9528e-01,  9.7527e-03,  ...,  9.8758e-01,
           1.5490e-01,  9.8793e-01]],

        [[ 7.8506e-01,  6.1943e-01, -8.2824e-01,  ...,  9.8756e-01,
           1.5500e-01,  9.8792e-01]],

        [[ 9.4540e-01, -3.2592e-01, -9.2457e-01,  ...,  9.8755e-01,
           1.5510e-01,  9.8790e-01]]])), ('encoder.0.weight', tensor([[-0.0027, -0.0104,  0.0077,  ..., -0.0128, -0.0048,  0.0116],
        [-0.0114,  0.0136, -0.0028,  ..., -0.0074, -0.0332,  0.0067],
        [-0.0174, -0.0208, -0.0382,  ...,  0.0023, -0.0410,  0.0140],
        ...,
        [-0.0089,  0.0364,  0.0464,  ...,  0.0295, -0.0050, -0.036

In [74]:
import scanpy as sc
adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/avg_counts/RMC-6236_block9051.h5ad")


In [75]:
adata

AnnData object with n_obs × n_vars = 1 × 62710
    obs: 'drug', 'block_id'

## uce embeddings

In [6]:
rm -rf /content/UCE

In [7]:
%cd /content

/content


In [8]:
# First, clone the repository
!git clone https://github.com/snap-stanford/UCE.git
%cd UCE



Cloning into 'UCE'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 227 (delta 62), reused 52 (delta 44), pack-reused 140 (from 1)
Receiving objects: 100% (227/227), 328.67 KiB | 14.29 MiB/s, done.
Resolving deltas: 100% (119/119), done.
/content/UCE


In [10]:
import scanpy as sc

adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/avg_counts_control/control.h5ad")
print(adata.var.index[:10])  # Print first 10 gene names


Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA'],
      dtype='object', name='gene_name')


In [11]:
!pip install mygene

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.7 MB/s eta 0:00:00


In [18]:
import scanpy as sc
import pandas as pd
import mygene

adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/avg_counts_control/control.h5ad")

mg = mygene.MyGeneInfo()
ensg_ids = [gene for gene in adata.var_names if gene.startswith("ENSG")]
query_result = mg.querymany(ensg_ids, scopes="ensembl.gene", fields="symbol", species="human")

ensg_to_symbol = {entry["query"]: entry.get("symbol", entry["query"]) for entry in query_result}
adata.var["gene_symbols"] = adata.var_names.map(lambda x: ensg_to_symbol.get(x, x))

# adata.write_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/avg_counts_control/control_updated.h5ad")


INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:querying 3001-4000 ...
INFO:biothings.client:querying 4001-5000 ...
INFO:biothings.client:querying 5001-6000 ...
INFO:biothings.client:querying 6001-7000 ...
INFO:biothings.client:querying 7001-8000 ...
INFO:biothings.client:querying 8001-9000 ...
INFO:biothings.client:querying 9001-10000 ...
INFO:biothings.client:querying 10001-11000 ...
INFO:biothings.client:querying 11001-12000 ...
INFO:biothings.client:querying 12001-13000 ...
INFO:biothings.client:querying 13001-14000 ...
INFO:biothings.client:querying 14001-15000 ...
INFO:biothings.client:querying 15001-16000 ...
INFO:biothings.client:querying 16001-17000 ...
INFO:biothings.client:querying 17001-18000 ...
INFO:biothings.client:querying 18001-19000 ...
INFO:biothings.client:querying 19001-20000 ...
INFO:biothings.client:querying 20001-20930 ...
INFO:biothings.client:Finished.
INF

In [21]:
adata.write_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/avg_counts_control/control_updated.h5ad")

In [22]:
control_adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/avg_counts_control/control_updated.h5ad")

ensg_mask = control_adata.var["gene_symbols"].str.startswith("ENSG")
ensg_indexes = control_adata.var.index[ensg_mask]

control_adata = control_adata[:, ~ensg_mask]



In [26]:
import os

In [36]:
import torch

human_embeddings_path = "model_files/protein_embeddings/Homo_sapiens.GRCh38.gene_symbol_to_embedding_ESM2.pt"
human_embeddings = torch.load(human_embeddings_path)

# Print a sample of stored gene keys
print("Example gene keys in embedding file:", list(human_embeddings.keys())[:10])


<ipython-input-36-457643d69bfe>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  human_embeddings = torch.load(human_embeddings_path)


Example gene keys in embedding file: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']


In [37]:
control_adata.var_names = control_adata.var["gene_symbols"]


/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:845: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "var")


In [38]:
control_adata.var_names

CategoricalIndex(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH',
                  'FUCA2', 'GCLC', 'NFYA',
                  ...
                  'SMG1P7-1', 'SMG1P5-1', 'ANKRD20A11P-1', 'COL6A4P1-1',
                  'PRSS30P-1', 'LNX2BP', 'POLGARF', 'LY6S', 'TMEM276-ZFTRAF1',
                  'TMEM276'],
                 categories=['5S_rRNA', '5S_rRNA-1', '5S_rRNA-2', '5S_rRNA-3', ..., 'hsa-mir-423', 'hsa-mir-1253', 'hsa-mir-8069-1', 'snoZ196'], ordered=False, dtype='category', name='gene_symbols', length=45584)

In [41]:
# Convert to lowercase to match embedding format
adata_gene_names = set(adata.var_names.str.lower())
embedding_gene_names = set(map(str.lower, human_embeddings.keys()))

# Find overlap
overlapping_genes = adata_gene_names.intersection(embedding_gene_names)
print(f"Number of overlapping genes: {len(overlapping_genes)}")


Number of overlapping genes: 19648


In [44]:
print(f"Total genes in adata: {len(control_adata.var_names)}")
print(f"Number of overlapping genes: {len(overlapping_genes)}")


Total genes in adata: 45584
Number of overlapping genes: 19648


In [42]:
control_adata.write_h5ad("/content/filtered_control.h5ad")


In [52]:
control_adata.var_names

CategoricalIndex(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH',
                  'FUCA2', 'GCLC', 'NFYA',
                  ...
                  'SMG1P7-1', 'SMG1P5-1', 'ANKRD20A11P-1', 'COL6A4P1-1',
                  'PRSS30P-1', 'LNX2BP', 'POLGARF', 'LY6S', 'TMEM276-ZFTRAF1',
                  'TMEM276'],
                 categories=['5S_rRNA', '5S_rRNA-1', '5S_rRNA-2', '5S_rRNA-3', ..., 'hsa-mir-423', 'hsa-mir-1253', 'hsa-mir-8069-1', 'snoZ196'], ordered=False, dtype='category', name='gene_symbols', length=45584)

In [65]:
# Run evaluation on control dataset
!python eval_single_anndata.py --adata_path "/content/filtered_control.h5ad" --dir "./control_output" --model_loc "minwoosun/uce-100m" --species "human"


ANNDATA PROCESSOR DONE...
Proccessing filtered_control
species human
adata shape before load gene embedding adata function (0, 0)
genes with embeddings 19790
Total genes with embeddings: 19790
First few genes in adata.var_names: []
Overlap between adata genes and embedding genes: 0
genes to use 0
Overlap between adata genes and embedding genes: 0
Genes actually kept in adata: 0
Traceback (most recent call last):
  File "/content/UCE/eval_single_anndata.py", line 158, in <module>
    main(args, accelerator)
  File "/content/UCE/eval_single_anndata.py", line 84, in main
    processor.preprocess_anndata()
  File "/content/UCE/evaluate.py", line 94, in preprocess_anndata
    process_raw_anndata(self.row,
  File "/content/UCE/data_proc/data_utils.py", line 214, in process_raw_anndata
    dataset, adata = anndata_to_sc_dataset(ad, species=species, labels=labels, covar_col=covar_col, hv_genes=None)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# Process first 5 drug files
for drug_file in drug_files[:5]:
    drug_path = os.path.join(drug_dir, f"filtered_{drug_file}")
    drug_name = drug_file.split('_')[0]
    output_dir = f"./drug_output/{drug_name}"
    os.system(f'python eval_single_anndata.py --adata_path "{drug_path}" --dir "{output_dir}" --model_loc "minwoosun/uce-100m"')

## scgpt embeddings

In [9]:
!ls "/content/drive/MyDrive/Colab Notebooks/esm cell state/h5ad"

plate3_10percent.h5ad  plate3_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad


In [10]:
import scanpy as sc

In [8]:
adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/h5ad/plate3_10percent.h5ad")
adata

AnnData object with n_obs × n_vars = 470494 × 62710
    obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate'

In [11]:
adata.obs

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,
01_094_002-lib_1072,smp_1687,774,924,1137,"[('Infigratinib', 5.0, 'uM')]",Infigratinib,CVCL_1547,lib_1072,01_094_002,0.008658,-0.023810,0.077473,G2M,full,NCI-H23,plate3
01_154_050-lib_1033,smp_1687,1864,2492,2972,"[('Infigratinib', 5.0, 'uM')]",Infigratinib,CVCL_0293,lib_1033,01_154_050,0.026886,0.054286,-0.103297,S,full,HEC-1-A,plate3
01_059_130-lib_1074,smp_1687,516,684,827,"[('Infigratinib', 5.0, 'uM')]",Infigratinib,CVCL_1478,lib_1074,01_059_130,0.002924,-0.027778,-0.023810,G1,minimal,NCI-H1573,plate3
01_030_092-lib_1027,smp_1687,2016,3044,3645,"[('Infigratinib', 5.0, 'uM')]",Infigratinib,CVCL_1517,lib_1027,01_030_092,0.017740,-0.107143,-0.158654,G1,full,NCI-H2030,plate3
01_027_080-lib_1096,smp_1687,1677,2372,2830,"[('Infigratinib', 5.0, 'uM')]",Infigratinib,CVCL_0131,lib_1096,01_027_080,0.026560,-0.040125,0.102747,G2M,full,A-172,plate3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96_163_049-lib_1077,smp_1782,580,697,851,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0480,lib_1077,96_163_049,0.060258,-0.000381,0.038830,G2M,minimal,PANC-1,plate3
95_124_186-lib_1089,smp_1781,1279,1823,2119,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0397,lib_1089,95_124_186,0.042238,-0.032571,-0.157143,G1,full,LS 180,plate3
95_098_128-lib_1064,smp_1781,486,577,672,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0399,lib_1064,95_098_128,0.032929,-0.043825,-0.004579,G1,minimal,LoVo,plate3


In [12]:
adata.obs['batch_id'] = adata.obs['plate']


In [1]:
!git clone https://github.com/bowang-lab/scGPT-spatial
%cd scGPT-spatial/

Cloning into 'scGPT-spatial'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 78 (delta 29), reused 40 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 1.70 MiB | 40.43 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/scGPT-spatial


In [2]:
# Specifically for Google Colab, install dependencies and download data

import os
import sys

if "google.colab" in sys.modules:
    print("Running on Google Colab")
    print("Installing dependencies...")
    !pip install -U scgpt
    # the optional dependency of flash-attion is skipped on colab
    !pip install wandb louvain

    # # NOTE: May need to restart runtime after the installation

    print("Downloading data and model ckpt...")
    !pip install -q -U gdown
    import gdown

    data_dir = "../../data"
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    if not os.path.exists(os.path.join(data_dir, "Kim2020_Lung.h5ad")):
        !gdown https://drive.google.com/uc?id=1z_0vWYMhRuRiD1EyhuFtY9ReIR0msWaL -O $data_dir/
    if not os.path.exists(os.path.join(data_dir, "covid_subsampled.h5ad")):
        !gdown https://drive.google.com/uc?id=1eD9LbxNJ35YUde3VtdVcjkwm-f4iyJ6x -O $data_dir/
    if not os.path.exists(os.path.join(data_dir, "ms")):
        gdown.download_folder(
            "https://drive.google.com/drive/folders/1Qd42YNabzyr2pWt9xoY4cVMTAxsNBt4v",
            output=os.path.join(data_dir, "ms"),
        )

    print("Downloading model ckpt...")
    model_dir = "../../save/scGPT_human"
    if not os.path.exists(model_dir):
        !mkdir -p $model_dir
        gdown.download_folder(
            "https://drive.google.com/drive/folders/1oWh_-ZRdhtoGQ2Fw24HP41FgLoomVo-y",
            output=model_dir,
        )

Running on Google Colab
Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of scgpt to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install scgpt==0.1.3 and scgpt==0.1.5 because these package versions have conflicting dependencies.

The conflict is caused by:
    scgpt 0.1.5 depends on llvmlite<0.39.0 and >=0.38.0
    scgpt 0.1.3 depends on llvmlite<0.39.0 and >=0.38.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
  

Retrieving folder contents


Processing file 1Vp51xYAOEu5_Wd7lilnatvMJ7tr2bsbl .DS_Store
Processing file 1bV1SHKVZgkcL-RmmuN51_IIUJTSJbXOi c_data.h5ad
Processing file 1casFhq4InuBNhJLMnGebzkRXM2UTTeQG filtered_ms_adata.h5ad


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1Vp51xYAOEu5_Wd7lilnatvMJ7tr2bsbl
To: /data/ms/.DS_Store
100%|██████████| 6.15k/6.15k [00:00<00:00, 9.96MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bV1SHKVZgkcL-RmmuN51_IIUJTSJbXOi
To: /data/ms/c_data.h5ad
100%|██████████| 20.6M/20.6M [00:00<00:00, 48.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1casFhq4InuBNhJLMnGebzkRXM2UTTeQG
To: /data/ms/filtered_ms_adata.h5ad
100%|██████████| 47.1M/47.1M [00:01<00:00, 42.4MB/s]


Download completed
Retrieving folder contents


Processing file 1hh2zGKyWAx3DyovD30GStZ3QlzmSqdk1 args.json
Processing file 14AebJfGOUF047Eg40hk57HCtrb0fyDTm best_model.pt
Processing file 1H3E_MJ-Dl36AQV6jLbna2EdvgPaqvqcC vocab.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1hh2zGKyWAx3DyovD30GStZ3QlzmSqdk1
To: /save/scGPT_human/args.json
100%|██████████| 1.30k/1.30k [00:00<00:00, 4.96MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=14AebJfGOUF047Eg40hk57HCtrb0fyDTm
From (redirected): https://drive.google.com/uc?id=14AebJfGOUF047Eg40hk57HCtrb0fyDTm&confirm=t&uuid=19afc246-9d6f-4434-8e88-a18cc3d255fe
To: /save/scGPT_human/best_model.pt
100%|██████████| 205M/205M [00:03<00:00, 59.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1H3E_MJ-Dl36AQV6jLbna2EdvgPaqvqcC
To: /save/scGPT_human/vocab.json
100%|██████████| 1.32M/1.32M [00:00<00:00, 182MB/s]
Download completed


In [4]:
!pip install numpy

In [4]:
!pip install --no-cache-dir "flash-attn<1.0.5"
!pip install pandas==1.3.5 scanpy==1.9.1 scikit-misc==0.1.4
!pip install llvmlite<0.39.0 numba<0.56.0 leidenalg<0.9.0 scvi-tools<0.17.0 datasets<3.0.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached pandas-1.3.5.tar.gz (4.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scanpy-1.9.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached scikit-misc-0.1.4.tar.gz (852 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
 

In [ ]:
!pip install scgpt


In [1]:
import scgpt
print("scGPT successfully installed!")


ModuleNotFoundError: No module named 'scgpt'

In [8]:
from pathlib import Path
import warnings

import scanpy as sc
import scib
import numpy as np
import sys

sys.path.insert(0, "../")

import scgpt as scg
import matplotlib.pyplot as plt
import anndata

plt.style.context('default')
warnings.simplefilter("ignore", ResourceWarning)

model_dir = Path("../../save/scGPT_human")

/usr/local/lib/python3.11/dist-packages/scgpt/model/model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")


ModuleNotFoundError: No module named 'torchtext'

In [ ]:
from flash_attn.modules.mha import FlashMHA


In [4]:
!pip show flash-attn


Name: flash_attn
Version: 2.7.4.post1
Summary: Flash Attention: Fast and Memory-Efficient Exact Attention
Home-page: https://github.com/Dao-AILab/flash-attention
Author: Tri Dao
Author-email: tri@tridao.me
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: einops, torch
Required-by: 


In [3]:
from flash_attn.flash_attention import FlashMHA

ModuleNotFoundError: No module named 'flash_attn.flash_attention'

In [9]:
from scgpt_spatial.tasks.cell_emb import embed_data

model_dir = "/content/drive/MyDrive/Colab Notebooks/esm cell state/scGPT_spatial_v1"
gene_col = "feature_name"
cell_type_col = "celltype"
batch_id_col = "batch_id"

ref_embed_adata = embed_data(
    adata,
    model_dir,
    gene_col=gene_col,
    obs_to_save=cell_type_col,
    batch_size=64,
    return_new_adata=True,
)

ref_embed_adata.obsm["X"] = ref_embed_adata.X.copy()
ref_embed_adata.obs["batch_id"] = adata.obs["batch_id"]


ModuleNotFoundError: No module named 'flash_attn.flash_attention'

## scAce embeddings

In [1]:
import scanpy as sc

In [2]:
adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/esm cell state/h5ad/plate3_10percent.h5ad")

In [3]:
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=200)
adata.raw = adata.copy()

sc.pp.normalize_per_cell(adata)
adata.obs['scale_factor'] = adata.obs.n_counts / adata.obs.n_counts.mean()

sc.pp.log1p(adata)

<ipython-input-3-ade24781ed1f>:5: FutureWarning: Use sc.pp.normalize_total instead
  sc.pp.normalize_per_cell(adata)
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_simple.py:588: FutureWarning: Use sc.pp.normalize_total instead
  normalize_per_cell(


In [6]:
# !python -m venv /content/scace_env

Error: Command '['/content/scace_env/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.


In [7]:
!pip install --no-cache-dir --upgrade --force-reinstall scikit-learn
!pip install --no-cache-dir --upgrade --force-reinstall scace


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 183.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 303.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 171.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 235.5 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.5.0
    Uninstalling threadpoolctl-3.5.0:
      Successfully uninstalled threadpoolctl-3.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  Attempting uninstall: scipy
    

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 251.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 237.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import h5py
import numpy as np
import scanpy as sc
import torch
import random
from scace import run_scace

seed = 666
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

adata = run_scace(adata, return_all=True)
